In [15]:
import os
import sys

import pandas as pd
import numpy as np
import random
import itertools

from matplotlib import pyplot as plt

from tqdm import tqdm

from pandarallel import pandarallel

import pymorphy2
import nltk
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from keras.preprocessing.sequence import pad_sequences

import gensim
from gensim.models.callbacks import CallbackAny2Vec
from gensim.models.phrases import Phrases, Phraser

import time


SEED = 1
def init_random_seed(value=0):
    random.seed(value)
    np.random.seed(value)
    torch.manual_seed(value)
    torch.cuda.manual_seed(value)
    torch.backends.cudnn.deterministic = True
init_random_seed(SEED)
    
pd.set_option('display.max_colwidth', 255)
tqdm.pandas()
pandarallel.initialize(progress_bar=True, nb_workers=8, use_memory_fs=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [43]:
abbr = pd.read_csv("../data/abbr.csv")
lenta_train = pd.read_csv("../data/lenta_train.csv")
lenta_test = pd.read_csv("../data/lenta_test.csv")

In [3]:
id2desc = {}
id2abbr = {}
for abbr, abbr_id, desc in abbr[["abbr_norm", "abbr_id", "desc_norm"]].values:
    id2desc[abbr_id] = desc
    id2abbr[abbr_id] = abbr

In [4]:
lenta_train["labels_new"] = (
    lenta_train["labels_new"].str.replace("W", "")
                            .str.replace("B", "")
                            .str.replace("E", "")
                            .str.replace("-", "")
)
train_texts = list(map(lambda x: x.split(" "), lenta_train["text_new"].to_list()))
train_labels = list(map(lambda x: x.split(" "), lenta_train["labels_new"].to_list()))

lenta_test["labels_new"] = (
    lenta_test["labels_new"].str.replace("W", "")
                            .str.replace("B", "")
                            .str.replace("E", "")
                            .str.replace("-", "")
)
test_texts = list(map(lambda x: x.split(" "), lenta_test["text_new"].to_list()))
test_labels = list(map(lambda x: x.split(" "), lenta_test["labels_new"].to_list()))

In [5]:
PAD_TOKEN = "<PAD>"
PAD_TOKEN_ID = 0

PAD_LABEL = "<NOLABEL>"
PAD_LABEL_ID = 0

EMPTY_LABEL = "_"
EMPTY_LABEL_ID = 1

train_texts_global = list(itertools.chain(*train_texts))
train_labels_global = list(itertools.chain(*train_labels))
train_labels_global = list(filter(lambda x: x != EMPTY_LABEL, train_labels_global))

UNIQUE_TOKENS = [PAD_TOKEN] + list(set(train_texts_global))
UNIQUE_LABELS = [PAD_LABEL, EMPTY_LABEL] + list(set(train_labels_global))

token2id = {label: i for i, label in enumerate(UNIQUE_TOKENS)}
id2token = {i: label for label, i in token2id.items()}

label2id = {label: i for i, label in enumerate(UNIQUE_LABELS)}
id2label = {i: label for label, i in label2id.items()}

MAX_SENTENCE_LEN = lenta_train.text_new.str.split(" ").str.len().max()
train_size = len(train_texts)
test_size = len(test_texts)
TOKENS_NUM = len(UNIQUE_TOKENS)
LABELS_NUM = len(UNIQUE_LABELS)

print(train_size, test_size, TOKENS_NUM, LABELS_NUM)

548700 137175 588317 5611


## 2. Обучение модели 

In [26]:
class EpochLogger(CallbackAny2Vec):
    '''Callback to log information about training'''

    def __init__(self, verbose: int = 1):
        self.epoch = 0
        self.verbose = verbose
        self.start_sec = int(time.time())
        self.last_loss = float("-inf")
        
    def on_epoch_end(self, model):
        if self.epoch % self.verbose == 0:
            loss = model.get_latest_training_loss()
            
            end_sec = int(time.time())
            diff_sec = end_sec - self.start_sec
            self.start_sec = end_sec
            
            print(f'Epoch {self.epoch}, loss diff {loss - self.last_loss}, sec {diff_sec}\n')
            
            self.last_loss = loss
        self.epoch += 1

In [27]:
epoch_logger = EpochLogger(verbose=1)

model = gensim.models.Word2Vec(
    sentences=train_texts, 
    vector_size=100, window=5, min_count=5, workers=16,
    sg=1, epochs=20, callbacks=[epoch_logger], compute_loss=True)

Epoch 0, loss diff inf, sec 37

Epoch 1, loss diff 25957900.0, sec 31

Epoch 2, loss diff 2504876.0, sec 29

Epoch 3, loss diff 1679120.0, sec 30

Epoch 4, loss diff 1770656.0, sec 30

Epoch 5, loss diff 1701768.0, sec 31

Epoch 6, loss diff 1453864.0, sec 29

Epoch 7, loss diff 1680000.0, sec 30

Epoch 8, loss diff 1653448.0, sec 30

Epoch 9, loss diff 1702064.0, sec 29

Epoch 10, loss diff 1740192.0, sec 30

Epoch 11, loss diff 1540160.0, sec 30

Epoch 12, loss diff 1144664.0, sec 30

Epoch 13, loss diff 1379280.0, sec 29

Epoch 14, loss diff 1537328.0, sec 30

Epoch 15, loss diff 1336552.0, sec 29

Epoch 16, loss diff 1495032.0, sec 30

Epoch 17, loss diff 1587208.0, sec 30

Epoch 18, loss diff 1445016.0, sec 30

Epoch 19, loss diff 1277272.0, sec 30



In [61]:
sum_n = 0
n = 0
for abbr_norm, desc_true in tqdm(abbr[abbr.desc_len == 1][["abbr_norm", "desc_norm"]].values):
    try:
        desc_pred = model.wv.most_similar(abbr_norm, topn=1)[0][0]
        if desc_true == desc_pred:
            sum_n += 1
        else:
            print(abbr_norm, desc_true, desc_pred)
    except:
        pass
    n += 1
    
sum_n / n

  3%|▎         | 71/2641 [00:00<00:07, 360.01it/s]

рост ростислав снижение
тайв тайваньский тайвань
кл клавиатура клуб
рыба рыболовство лосось
мб мегабайт быть
мег мегабайт мбайт
кол колония количество
мв микроволновый видеофон
год летник 2005
расти российский россия
ала алава 31фп
широта штурмовик школа
мол молекулярный молочный
бол болезнь большой
нп нефтепровод населить
зап запасный запад
арт артемий артист
охота охота изюбрь
муза музыка музыкальный
ст станица статья
стан станица станислав
холод холодильник тёплый
святой святослав cвятого
преим преимущественно преимущество
ак акустика барс
далее дорожный дом
экс экспортный быв
рука рукоятка руководитель
св свеча свз
деп депутатский депутат
ккб казкоммерцбанк générale


  4%|▍         | 108/2641 [00:00<00:07, 344.92it/s]

мама мама бабушка
пп противопехотный пункт
фэ фотоэлемент диэлектрик
ижм ижма нижнеангарска
дорев дореволюционный доревол
тип типография валрус
гора горизонт город
ограничить ограничительный ограничивать
чрс черский булатниковском
пан панама панель
инд индивидуальный sensex
ник никарагуа николай
шах шахтёрск шахта
рс радиосвязь рация
мкм микрометр нанометр
опт оптовый оптический
омск омский новокузнецк
штурм штурмовик штурмовать
бакт бактериолог бактериологический
спец специализировать специалист
всё всероссийский таки
тлгр телеграф тлг
марка марокканский бренд
кислый кислый кислота
окт октав октябрь
орг оргтехника организация
зах захар заход
ткб транскредитбанк промсвязьбанк
кр красный крупный
эт эталон этаж
пул пулемётный якб
пулем пулемётный артиллерийский
почтить почтительный увековечить
мак макар мая
тупой тупик вонзиться
гора горный город
бол боливия большой
душа душанбе утолить


  7%|▋         | 185/2641 [00:00<00:07, 346.11it/s]

...

 99%|█████████▉| 2620/2641 [00:07<00:00, 357.00it/s]

докл докладчик доклад
уф ультрафиолет ультрафиолетовый
право праведный вправе
раст раствор растение
илл иллюстратор иллюстрация
ап авиапредприятие администрация
кол колебание количество
вк военкомат волк
цвет цветной цв
ориг оригинал оригинальный
птица птицеводство насекомое
орг органический организация
дж джек джон
дв двор arrius
лита литейный литература
корд кордова manuel
уна удмуртнефть унсо
пр проспект проект
сол соломон солист
ар арагон арабский
упр управление управл
сп спирт спорт
универ университет унив
каз казачий казахстан
слово словарь сл
уста устойчивость установка
ар арык арабский
ива иваново иван
дол долина доллар
рбк росбизнесконсалтинг коммерсант
популяр популярный попул
контры контрольный ревизионный
ст станция статья


100%|██████████| 2641/2641 [00:07<00:00, 332.58it/s]

ла лавочкин риоха
эш эшелон шугур
ас ассоциация ассоц
кр кыргызстан крупный
фин финляндия финанс
мех механизировать оторочить
фил философия филиал
аца автоцистерна аналитический
инстр инструкция инструмент


0.5327527451722832

## 3. Обучение модели на n-граммах 

In [66]:
from nltk.corpus import stopwords
stopwords.words("russian")[:5]

['и', 'в', 'во', 'не', 'что']

In [87]:
phrase_model = Phrases(train_texts, 
                       min_count=1, 
                       threshold=10,
                       connector_words=stopwords.words("russian"))

In [92]:
TOKENS_NUM, len(phrase_model.vocab)

(588317, 21900910)

In [93]:
phrase_model = phrase_model.freeze()

In [94]:
train_texts_frozen = phrase_model[train_texts]

In [96]:
epoch_logger = EpochLogger(verbose=1)

model = gensim.models.Word2Vec(
    sentences=train_texts_frozen, 
    vector_size=100, window=5, min_count=5, workers=16,
    sg=1, epochs=20, callbacks=[epoch_logger], compute_loss=True)

Epoch 0, loss diff inf, sec 120

Epoch 1, loss diff 14590464.0, sec 72

Epoch 2, loss diff 10818652.0, sec 71

Epoch 3, loss diff 10565660.0, sec 71

Epoch 4, loss diff 3265208.0, sec 71

Epoch 5, loss diff 632312.0, sec 74

Epoch 6, loss diff 612240.0, sec 72

Epoch 7, loss diff 664848.0, sec 71

Epoch 8, loss diff 710712.0, sec 71

Epoch 9, loss diff 699752.0, sec 72

Epoch 10, loss diff 739760.0, sec 73

Epoch 11, loss diff 715160.0, sec 72

Epoch 12, loss diff 697784.0, sec 70

Epoch 13, loss diff 740048.0, sec 72

Epoch 14, loss diff 682744.0, sec 70

Epoch 15, loss diff 688736.0, sec 71

Epoch 16, loss diff 725208.0, sec 73

Epoch 17, loss diff 711176.0, sec 70

Epoch 18, loss diff 692952.0, sec 71

Epoch 19, loss diff 696120.0, sec 71



In [104]:
model.wv.most_similar("екб", topn=10)

[('нск', 0.8772291541099548),
 ('екатеринбург', 0.8657060265541077),
 ('челябинск', 0.8627609610557556),
 ('ёбург', 0.8323217034339905),
 ('новосиб', 0.8217184543609619),
 ('пермь', 0.8089326024055481),
 ('воронеж', 0.8049417734146118),
 ('нн', 0.804835319519043),
 ('нижний_новгород', 0.795190691947937),
 ('новгород', 0.7928652167320251)]

In [107]:
sum_n = 0
n = 0
for abbr_norm, desc_true in tqdm(abbr[["abbr_norm", "desc_norm"]].values):
    try:
        desc_pred = model.wv.most_similar(abbr_norm, topn=1)[0][0].replace("_", " ")
        if desc_true == desc_pred:
            sum_n += 1
        else:
            print(abbr_norm, desc_true, desc_pred)
    except:
        pass
    n += 1
    
sum_n / n

  0%|          | 5/5625 [00:00<02:10, 43.00it/s]

замгенпрокурора заместитель генеральный прокурор кехлеров
быт бытовой зримый
рс реактивный снаряд радиостанция
мц морской цель урология
сковс северо кавказский окружный военный суд ходатайство обвинять
мфо международный финансовый организация заёмщик
ржд российский железный дорога оао ржд
немой немецкий чей


  0%|          | 15/5625 [00:00<02:05, 44.86it/s]

валд валдайский форарльберг
рост ростислав темп рост
фольклорн фольклорный анс нары
тайв тайваньский тайвань
кл клавиатура клуб
клава клавиатура hdmi
пк пивоваренный компания пресс конференция
мц материальный ценность урология


  0%|          | 25/5625 [00:00<02:02, 45.70it/s]

...

100%|█████████▉| 5614/5625 [02:04<00:00, 45.26it/s]

кк контрольный комиссия красноярский край
гвин гвинейский maersk line
спецслужба специальный служба контрразведка
сс специальный служба село
тд торговый дом предприятие войти
фил философия уфимский государственный
аца автоцистерна подготовить сотрудник


100%|██████████| 5625/5625 [02:04<00:00, 45.15it/s]

вак вакансия кол бюджетный
цве центральный и восточный европа наиб динамично
сцве центральный и восточный европа oesterreichische volksbanken
ак авиационный комплекс 400 граммовый
турпоток туристический поток поток гость
го городской отделение октябрь 2016
нао ненецкий автономный округ ненецкий автономный
инстр инструкция инструмент
гост гостиница отель


0.22311111111111112